In [ ]:
import numpy as np
import tflite_runtime.interpreter as tflite
import cv2
import time
import tkinter as tk
from tkinter import messagebox
from pca9685_driver import Device

device = Device(0x40)
device.set_pwm_frequency(50)

#move the arm to target angles
def move_to_angles(base_angle, shoulder_angle, elbow_angle):
    # Convert angles to servo positions
    servo_positions = [int(angle * 180 / np.pi * (4096 / 360)) for angle in [base_angle, shoulder_angle, elbow_angle]]

    for i, position in enumerate(servo_positions, start=4):
        device.set_pwm(i, 0, position)

# Load model
model_path = r"C:\Users\aliha\Desktop\FYP\Final\tumor_detection_model.tflite"  
interpreter = tflite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Open camera
cap = cv2.VideoCapture(0)

# Defining a threshold for confidence (you can adjust this)
threshold = 0.1

while True:
    
    ret, frame = cap.read()

    input_image = cv2.resize(frame, (224, 224))  
    input_image = input_image.astype(np.float32) / 255.0  
    input_image = np.expand_dims(input_image, axis=0)  

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_image)

    # Run inference
    interpreter.invoke()

    # Get the output tensor
    output = interpreter.get_tensor(output_details[0]['index'])

    # Post-process the output
    prediction = np.argmax(output)
    confidence = output[0][prediction]

    # Make the prediction based on the threshold
    result = "Malignant" if confidence > threshold else "Benign"

    
    if result == "Malignant":
        cv2.putText(frame, "Malignant Tumor Detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow("Tumor Detection", frame)

        root = tk.Tk()
        root.withdraw()
        messagebox.showinfo("Tumor Detected", "The tumor is malignant!")

        base_angle = np.pi / 2
        shoulder_angle = np.pi / 2
        elbow_angle = np.pi / 2

        move_to_angles(base_angle, shoulder_angle, elbow_angle)

        time.sleep(2)

        # Move the servo motors to 180 degrees
        shoulder_angle = np.pi
        elbow_angle = np.(5*pi/6)
        move_to_angles(base_angle, shoulder_angle, elbow_angle)
        
        # Move the servo motors to 180 degrees
        shoulder_angle = np.pi
        elbow_angle = np.pi
        move_to_angles(base_angle, shoulder_angle, elbow_angle)
        
        # Move the servo motors to 180 degrees
        shoulder_angle = np.pi
        elbow_angle = np.(5*pi/6)
        move_to_angles(base_angle, shoulder_angle, elbow_angle)


    else:
        cv2.putText(frame, f"Benign Tumor Detected! Confidence: {confidence * 100:.2f}%", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("Tumor Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()
